In [33]:
import pygal
import numpy as np
import pandas as pd
import math
import os
from scipy.stats import ks_2samp
from scipy.stats import chisquare
from IPython.display import display, HTML
%matplotlib inline

base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [34]:
df=pd.read_csv('./csv/movies.csv')

In [35]:
df.head()

,movie_title,movie_imdb_link,color,genre_4,duration,gross,genre_1,genre_2,genre_3,num_voted_users,facenumber_in_poster,language,country,content_rating,title_year,imdb_score
0,Femme Fatale,http://www.imdb.com/title/tt0280665/?ref_=fn_t...,Color,Thriller,114.0,NaN,Crime,Drama,Mystery,28584,1,English,France,R,2002.0,6.3
1,An Unfinished Life,http://www.imdb.com/title/tt0350261/?ref_=fn_t...,Color,NaN,108.0,NaN,Drama,NaN,NaN,24033,3,English,USA,PG-13,2005.0,7.0
2,C.H.U.D.,http://www.imdb.com/title/tt0087015/?ref_=fn_t...,Color,NaN,96.0,NaN,Horror,Sci-Fi,NaN,7528,0,English,USA,R,1984.0,5.5
3,Just Wright,http://www.imdb.com/title/tt1407061/?ref_=fn_t...,Color,NaN,100.0,21520719.0,Comedy,Romance,Sport,10103,1,English,USA,PG,2010.0,5.8
4,Yentl,http://www.imdb.com/title/tt0086619/?ref_=fn_t...,Color,NaN,132.0,30400000.0,Drama,Musical,Romance,9503,1,English,UK,PG,1983.0,6.6


### Analisis Exploratorio de Datos

### a) Indique cuales de las variables presentadas son discretas y cuales continuas.

| Variable      | Cont/Disc     |
| ------------- |:-------------:|
| movie_title   | Discreto |
| movie_imdb_link| Discreto  |
| color | Discreto|
| duration | Continuo|
| gross | Continuo|
| genre_1 | Discreto|
| genre_2 | Discreto|
| genre_3 | Discreto|
| genre_4 | Discreto|
| num_voted_users | Continuo|
| facenumber_in_poster | Continuo|
| language | Discreto|
| country | Discreto|
| content_rating | Discreto|
| title_year | Discreto|
| imdb_score | Continuo|


### b) Realice una exploración visual ligera de los datos. Independientemente de la herramienta (Tableau, Pygal), agregue imágenes que respalden la exploración

In [36]:
# eliminamos este registro por que tiene esta cadena ' Quite a Conundrum\xa0'
df=df.drop([3319]).reset_index(drop=True)

In [37]:
# ya que solo 12 registros presentan nan values los vamos a eliminar
df=df.dropna(subset=['facenumber_in_poster'])

In [38]:
#convertimos las columnos a tipo int
df["num_voted_users"]=df["num_voted_users"].astype(int)
df["facenumber_in_poster"]=df["facenumber_in_poster"].astype(int)
df["title_year"]=df["title_year"].astype(str)

In [39]:
def plot_histogram(df,var):
    freq, bins = np.histogram(df[var])

    hist = list(zip(freq, bins, bins[1:]))

    hist_plot = pygal.Histogram()

    hist_plot.add(var, hist)
    display(HTML(base_html.format(rendered_chart=hist_plot.render(is_unicode=True))))

In [40]:
def plot_stackbar(df,val,route=""):
    line_chart = pygal.StackedBar()
    line_chart.title = val
    line_chart.x_labels = [val]
    aux = df[val].value_counts()
    for i in range(len(aux.values)):
            line_chart.add(str(aux.index[i]), aux.values[i])
    if route!='':
        return line_chart.render_to_png(route)
    display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))

In [41]:
def plot_bar(df,val,route=""):    
    line_chart = pygal.Bar()
    line_chart.title = val
    aux = df[val].value_counts()
    for i in range(len(aux.values)):
        line_chart.add(str(aux.index[i]), aux.values[i])
    if route!='':
        return line_chart.render_to_png(route)
    display(HTML(base_html.format(rendered_chart=line_chart.render(is_unicode=True))))

In [42]:
def plot_hist(df,value,route=''):
    info=df[value].describe()
    r=info["max"]-info["min"]
    k=math.ceil(1+3.322*math.log10(info["count"]))
    a=int(round(r/k))
    ranges=[]
    for i in range(int(info["min"]),int(info["max"]+2),a):
        ranges.append(i)
    aux=np.histogram(df[value], bins=ranges)
    histograma=[]
    for i in range(len(aux[0])):
        histograma.append([aux[0][i],aux[1][i],aux[1][i+1]])
    hist = pygal.Histogram()
    hist.add(value,histograma)
    if route!='':
        return hist.render_to_png(route)
    display(HTML(base_html.format(rendered_chart=hist.render(is_unicode=True))))

In [43]:
def split_variables(df):
    ls_con=[]
    ls_dis=[]
    ls_date=[]
    for i in range(len(df.columns)):
        if df.dtypes[i] == 'object':
            ls_dis.append(df.columns[i])
        elif df.dtypes[i] == 'int64' or df.dtypes[i]=='float64':
            ls_con.append(df.columns[i])
        elif df.dtypes[i] == 'datetime64[ns]':
            ls_date.append(df.columns[i])
    return ls_con,ls_dis,ls_date

In [44]:
ls_con,ls_dis,ls_date=split_variables(df)

In [45]:
os.mkdir('./img/antes')

FileExistsError: [Errno 17] File exists: './img/antes'

In [46]:
for i in ls_con:
    plot_hist(df,i,"./img/antes/cont_{}.png".format(i))

In [47]:
for i in ls_dis[2:]:
    plot_stackbar(df,i,"./img/antes/disc_{}.png".format(i))

### c) Remueva outliers de aquellas variables que los presenten. Utilice cualquier método para ello. Muestre un cuadro con el número de registros antes y despúes de los tratamientos

In [48]:
def iqr(df,variable):
    desc = df[variable].describe()
    iqr_value= desc["75%"]- desc["25%"]
    bandaIzq = desc["25%"] - (1.5*iqr_value)
    bandaDer = desc["75%"] + (1.5*iqr_value)
    aux = df.copy()
    return aux[(aux[variable]>bandaIzq) & (aux[variable]<bandaDer)]

In [49]:
def percentiles(df,variable,percent):
    desc = df[variable].describe(percentiles=percent)
    aux = df.copy()
    return aux[(aux[variable]>desc["{}%".format(str(int(percent[0]*100)))]) & (aux[variable]<desc["{}%".format(str(int(percent[1]*100)))])]

In [50]:
def media(df,variable,n_sigma):
    media = df[variable].mean()
    sigma = df[variable].std()
    aux = df.copy()
    return aux[(media-n_sigma*sigma <= aux[variable])  & (aux[variable] <= media + n_sigma*sigma )]

In [51]:
df[ls_con[3]].describe(percentiles = [x/10 for x in range(10)]+[0.01, 0.05, 0.95, 0.99],)

count    5030.000000
mean        1.371173
std         2.013576
min         0.000000
0%          0.000000
1%          0.000000
5%          0.000000
10%         0.000000
20%         0.000000
30%         0.000000
40%         0.000000
50%         1.000000
60%         1.000000
70%         2.000000
80%         2.000000
90%         4.000000
95%         5.000000
99%         8.000000
max        43.000000
Name: facenumber_in_poster, dtype: float64

In [52]:
per_df=df.copy()

df=percentiles(per_df,ls_con[0],[0.01,0.99])

| Antes  | Despues|
| -------|:----:|
| 5030   | 4911 |

### d) Normalice las variables discretas que lo requieran. Muestre un cuadro con lasc ategorías resultantes por cada variable

### e) Realice una nueva exploraci´on visual, esta vez con el objetivo de ver los efectos de los tratamientos realizados. De igual modo, agregue las im´agenes correspondientes.

In [53]:
# os.mkdir('./img/despues')

for i in ls_con:
    plot_hist(df,i,"./img/despues/cont_{}.png".format(i))

for i in ls_dis[2:]:
    plot_stackbar(df,i,"./img/despues/disc_{}.png".format(i))

# Ingeniería de datos

### a) Cree una variable binaria que indique sí el score de la película es mayor al promedio. Analice gráficamente la frecuencia de los valores generados.

In [54]:
ls_con

['duration', 'gross', 'num_voted_users', 'facenumber_in_poster', 'imdb_score']

In [55]:
df["var1"]=df["duration"]/df["gross"]
df["var2"]=df["num_voted_users"]*df["imdb_score"]

# Tratamiento de valores ausentes

### a) Elimine aquellas columnas que superen el umbral de 70% o más de presencia de valores ausentes. Indique que columnas fueron eliminadas.

### c) Impute las variables continuas que lo requieran mediante el uso de la mediana. La variable imputada debe almacenarse en una variable nueva, dejando a la original intacta.

### d) Impute las variables continuas que lo requieran mediante el uso de la media. La variable imputada debe almacenarse en una variable nueva, dejando a la original intacta.

In [64]:
((df.isna().sum()*100)/4911).to_frame()

,0
movie_title,0.000000
movie_imdb_link,0.000000
color,0.325799
genre_4,71.736917
duration,0.000000
gross,69.721034
genre_1,0.000000
genre_2,12.482183
genre_3,39.401344
num_voted_users,0.000000


Se calculo el porcentaje de valores nulos de cada variable y la unica que supera el 70% es la variable `genre_4` entonces esta variable la eliminaremos


### b) Impute las variables discretas que lo requieran mediante el uso de la moda. La variable imputada debe almacenarse en una variable nueva, dejando a la original intacta.